Exporta el paquete `Simpletransformers` al notebook

In [ ]:
!pip install -q simpletransformers

Se definen los conjuntos de entrenamiento y test con tres posibles categorías:

* `0` - solicitud de información
* `1` - queja o reclamación
* `2` - recomendación



In [ ]:
corpus_train = [
  ['Quiero información sobre cómo contratar la línea', 0],
  ['Solicito ayuda de vuestros servicios', 0],
  ['Vengo a pedir atención sobre un producto', 0],
  ['Quisiera conocer más sibre ese servicio', 0],
  ['Quiero poner una queja', 1],
  ['Voy a hacer una reclamación por el mal servicio', 1],
  ['Vengo a poner una queja', 1],
  ['Estoy inconforme con el servicio y quiero que me atiendan ya', 1],
  ['Este producto es totalmente recomendable', 2],
  ['Le diré a mis amigos que lo compren', 2],
  ['Me encanta el producto y lo recomiendo', 2],
  ['Muy buen producto, la gente debería conocerlo más', 2],
]

corpus_eval = [
  ['Por favor, quiero ayuda sobre el uso', 0],
  ['Me gustaría poner una reclamación', 1],
  ['Muy recomendable sin duda', 2],
]

Preparamos los conjuntos

In [ ]:
import pandas as pd

train_df =  pd.DataFrame(corpus_train, columns=['text','labels'])
eval_df = pd.DataFrame(corpus_eval, columns = ["text", "labels"])

Importamos la librería y configuramos el log a nivel de Warnings

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
 
logging.basicConfig(level=logging.WARNING)

logging.getLogger('transformers').setLevel
(logging.WARNING)

30

Preparamos el modelo utilizando `distilbert-base-multilingual-cased`, un modelo de la variante de BERT llamado **DistilBERT**, que es multilingüe y ha sido entrenado con múltiples idiomas. El parámetro de `num_train_epochs` indica el número de iteraciones de entrenamiento sobre el conjunto de datos, en este caso, 20.

In [ ]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 15 # las iteraciones para Colab deben ser menores de 20, si no se llena el disco
model_args.overwrite_output_dir = True
model = ClassificationModel('distilbert', 'distilbert-base-multilingual-cased', args=model_args, num_labels=3) # para no usar use_cuda=False, hay que usar un entorno con GPU

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.

Entrenamos el modelo

In [ ]:
model.train_model(train_df)

  0%|          | 0/12 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/2 [00:00<?, ?it/s]

...

Running Epoch 11 of 15:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 12 of 15:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 13 of 15:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 14 of 15:   0%|          | 0/2 [00:00<?, ?it/s]

(30, 0.6907058715820312)

Evaluamos el modelo entrenado

In [ ]:
result, outputs, wrong = model.eval_model(eval_df)
print(result)

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

{'mcc': 0.6123724356957946, 'eval_loss': 0.7784016728401184}


Ahora vamos con las pruebas de clasificación

In [ ]:
corpus_test = ['Lo único que se puede hacer es una reclamación',          # correct tag 1
               'Qué bueno es esto, lo recomiendo',                        # correct tag 2
               'I want to make a complaint because the service is awful', # correct tag 1
               'Je veux déposer une plainte concernant le service']       # correct tag 1

predictions, outputs = model.predict(corpus_test)
print(predictions)
print(outputs)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[1 2 1 1]
[[-0.97753906  0.63574219 -0.12121582]
 [-1.16894531 -0.03683472  0.81005859]
 [-0.72851562  0.29248047  0.13842773]
 [-0.16210938  0.07324219 -0.16186523]]
